In [ ]:
import a1chemy.data_source as data_source
import a1chemy.indicators as indicators
import a1chemy.plot.plotly_tools as plotly_tools
import pymongo
import operator
import pandas as pd
from tqdm.notebook import tqdm
import base64

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)

In [ ]:
ticks = mongo_ticks_client.find_one(exchange='SH', symbol='SH510300')

In [ ]:
widgets.Text(value='Broad Market')

In [ ]:
import ipywidgets as widgets

image = plotly_tools.ticks_thumbnail(ticks=ticks, start=-20)

title=widgets.HBox([widgets.Text(value='Broad Market', layout=widgets.Layout(width='10%', height='80px')), widgets.Text(value='5D%'), widgets.Text(value='1M'), widgets.Text(value='shares')])
value1=widgets.HBox([widgets.Text(value='沪深300'), widgets.Text(value='5'), widgets.Image(value=image.data)])

widgets.VBox([title, value1])